In [11]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub  #to get models from tensorflow tools
import tensorflow_datasets as tfds #to get dataset from tensorflow tools

#first, we have to enable tensorflow to run eagerly, next(iter) need to be used in eager = true
tf.compat.v1.enable_eager_execution()
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")
#try changing train and test comparison by 80 and 20
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:80%]', 'train[80%:]', 'test'),
    as_supervised=True)
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch
train_labels_batch
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
#try changing the epoch to get better results
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=24,
                    validation_data=validation_data.batch(512),
                    verbose=1)
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

#epoch 24 with 60% train = acc : 0.853, loss : 0.318
#epoch 24 with 80% train = acc : 0.866, loss : 0.318 but with lower loss in the end of epoch for validation

Version:  1.15.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_7 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_12 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________
Epoch 1/24
40/40 [==============================] - 11s 267ms/step - loss: 0.7002 - acc: 0.5469 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/24
40/40 [==============================] - 9s 236ms/step - loss: 0.6346 - acc: 0.6101 - val_loss: 0.6116 - val_acc: 0.6236
Epoch 3/24
40/40 [==========